# Load data
- define a sample that passes QC and has data
- load the data for the modalities
- store the stack and the corresponding model

In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
# Paths
root_p = '/home/surchs/sim_big/PROJECT/abide_hps/'
# Pheno
abide1_p = os.path.join(root_p, 'pheno', 'abide_1_complete.csv')
# Data
ct_p = os.path.join(root_p, 'ct')
fc_p = os.path.join(root_p, 'fc')
# File templates
ct_t = '{}+{:07}_{}+{}_native_rms_rsl_tlaplace_30mm_left.txt'
fc_t = 'fmri_{:07}_session_1_run1.nii.gz'

In [3]:
pheno = pd.read_csv(abide1_p)

## Define sample
Criteria:
- `anat_qc` > 1
- `fc_qc` > _'Fail'_
- in psm sample
- data available for both ct and fc

In [8]:
sub_ind = [row['in_psm'] 
           and row['ct_available'] 
           and row['fc_available'] 
           and row['Ratings'] > 1 
           and not row['status']=='Fail'
           for rid, row in pheno.iterrows()]

In [9]:
np.sum(sub_ind)

370

In [10]:
sample = pheno[sub_ind]

In [11]:
sample['DX_GROUP'].value_counts()

Control    188
Autism     182
Name: DX_GROUP, dtype: int64

This seems like a good starting sample. I'd say we go with this

## Get the corresponding data
Criteria:
- put the vectorized CT information in a subject x vertex stack
- get or find seed maps for MIST_20
- vectorize them with the group mask and stack them according to the model